In [1]:
!python --version

Python 3.6.10


In [2]:
!which python

/opt/bitnami/python/bin/python


In [3]:
import findspark
findspark.init()

### Запуск на spark-master

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("MySparkApp") \
    .getOrCreate()


In [8]:
# Пример простого задания для выполнения на Spark
data = [("John", 28), ("Anna", 24), ("Mike", 32)]
columns = ["Name", "Age"]

# Создайте DataFrame
df = spark.createDataFrame(data, columns)

# Выведите данные
df.show()


+----+---+
|Name|Age|
+----+---+
|John| 28|
|Anna| 24|
|Mike| 32|
+----+---+



In [9]:
spark.stop()

### Запуск Spark с Kafka

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Создаем SparkSession
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("KafkaTest") \
    .getOrCreate()

# Схема для вложенного поля "value"
value_schema = StructType([
    StructField("schema", StructType([
        StructField("type", StringType(), True),
        StructField("optional", StringType(), True)
    ]), True),
    StructField("payload", StringType(), True)
])

# Чтение с самого начала топика
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka-cluster:9092") \
    .option("subscribe", "streaming-user-registration") \
    .option("startingOffsets", "earliest") \
    .option("maxOffsetsPerTrigger", "1") \
    .load()

# Преобразуем значение поля "value" в строку
df = df.selectExpr("CAST(value AS STRING) as json_value")

# Извлекаем вложенный JSON из поля "value"
df_parsed = df.withColumn("parsed_value", from_json(col("json_value"), value_schema))

# Извлекаем поле "payload", которое содержит нужные данные
df_payload = df_parsed.select("parsed_value.payload")

# Определяем схему для поля "payload"
payload_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("parent_profile", StructType([
        StructField("age", IntegerType(), True),
        StructField("gender", StringType(), True),
        StructField("location", StructType([
            StructField("country", StringType(), True),
            StructField("city", StringType(), True)
        ]), True),
        StructField("device", StructType([
            StructField("brand", StringType(), True),
            StructField("model", StringType(), True),
            StructField("os_version", StringType(), True)
        ]), True)
    ]), True),
    StructField("child_profile", StructType([
        StructField("age", IntegerType(), True),
        StructField("gender", StringType(), True),
        StructField("device", StructType([
            StructField("brand", StringType(), True),
            StructField("model", StringType(), True),
            StructField("os_version", StringType(), True)
        ]), True)
    ]), True)
])

# Преобразуем поле "payload" в структурированный формат
df_final = df_payload.withColumn("payload_data", from_json(col("payload"), payload_schema)) \
    .select("payload_data.*")


# Stream
#query = df_final.writeStream \
#    .outputMode("append") \
#    .format("console") \
#    .option("truncate", "false") \
#    .start()

# Функция для обработки каждого батча
def process_batch(df, epoch_id):
    df.show(truncate=False)  # Показываем батч

# Запускаем потоковую обработку с foreachBatch
query = df_final.writeStream \
    .outputMode("append") \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination()


+-------+------------------------------------------------------------+-----------------------------------------------+
|user_id|parent_profile                                              |child_profile                                  |
+-------+------------------------------------------------------------+-----------------------------------------------+
|11111  |[40, female, [USA, New York], [Google, Pixel 6, Android 12]]|[8, male, [Samsung, Galaxy S21, Android 13]]   |
|12346  |[40, female, [USA, New York], [Google, Pixel 6, Android 12]]|[8, male, [Samsung, Galaxy S21, Android 12]]   |
|12345  |[35, male, [Russia, Moscow], [Apple, iPhone 13, iOS 16.3]]  |[10, female, [Samsung, Galaxy A52, Android 11]]|
+-------+------------------------------------------------------------+-----------------------------------------------+

+-------+------------------------------------------------------------+--------------------------------------------+
|user_id|parent_profile                           

KeyboardInterrupt: 

In [17]:
spark.stop()